In [3]:
!pip3 install http://download.pytorch.org/whl/cu90/torch-0.4.0-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision


    100% |████████████████████████████████| 566.4MB 48.1MB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5d6d6000 @  0x7eff26aef1c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 4.0MB/s 
    100% |████████████████████████████████| 2.0MB 8.6MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [4]:
# Convolutional Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# Install Tensorflow from the website: https://www.tensorflow.org/versions/r0.12/get_started/os_setup.html

# Installing Keras
# pip install --upgrade keras

# Part 1 - Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(output_dim = 128, activation = 'relu'))
classifier.add(Dense(output_dim = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('C:\\Users\\SONAL BANSAL\\Desktop\\WORK\\Machine_Learning AZ\\Machine Learning A-Z Template Folder\\Part 8 - Deep Learning\\Section 40 - Convolutional Neural Networks (CNN)\\Convolutional_Neural_Networks\\dataset\\training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('C:\\Users\\SONAL BANSAL\\Desktop\\WORK\\Machine_Learning AZ\\Machine Learning A-Z Template Folder\\Part 8 - Deep Learning\\Section 40 - Convolutional Neural Networks (CNN)\\Convolutional_Neural_Networks\\dataset\\test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

classifier.fit_generator(training_set,
                         samples_per_epoch = 8000,
                         nb_epoch = 25,
                         validation_data = test_set,
                         nb_val_samples = 2000)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 3..., activation="relu")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1)`


FileNotFoundError: ignored

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

learning_rate = 0.001

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

 
# MNIST Dataset (Images and Labels)
train_dataset = dsets.MNIST(root ='./data',
                            train = True,
                            transform = transforms.ToTensor(),
                            download = True)
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = 10,
                                           shuffle = True)
test_dataset = dsets.MNIST(root ='./data',
                           train = False,
                           transform = transform)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = 10,
                                          shuffle = False)
 

In [28]:
print(len(train_dataset),len(test_dataset))

60000 10000


In [29]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 4 * 4, 
                             120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(train_loader)
# # create your optimizer
# optimizer = optim.SGD(net.parameters(), lr=0.01)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)


In [37]:
for epoch in range(5):  # loop over the dataset multiple times
    total=0
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(train_loader):
      
#     for i, data in enumerate(trainloader, 0):
#         # get the inputs
#         inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()
        total+=1
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            
            running_loss = 0.0

print('Finished Training')
print(total)

[1,  2000] loss: 0.010
[1,  4000] loss: 0.015
[1,  6000] loss: 0.010
[2,  2000] loss: 0.005
[2,  4000] loss: 0.010
[2,  6000] loss: 0.014
[3,  2000] loss: 0.007
[3,  4000] loss: 0.016
[3,  6000] loss: 0.010
[4,  2000] loss: 0.009
[4,  4000] loss: 0.012
[4,  6000] loss: 0.013
[5,  2000] loss: 0.005
[5,  4000] loss: 0.012
[5,  6000] loss: 0.014
Finished Training
6000


In [38]:
correct = 0
total = 0
pred=[]
label=[]
with torch.no_grad():
    for images, labels in test_loader:
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        pred.append(predicted)
        label.append(labels)
#         print(predicted.shape)
#         print(labels.shape)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print('Accuracy of the test images: %d %%' % (
    100 * correct / total))

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

print(total)
print(pred)
print(label)


Accuracy of the test images: 90 %
Accuracy of the test images: 90 %
Accuracy of the test images: 86 %
Accuracy of the test images: 90 %
Accuracy of the test images: 90 %
Accuracy of the test images: 90 %
Accuracy of the test images: 88 %
Accuracy of the test images: 87 %
Accuracy of the test images: 88 %
Accuracy of the test images: 89 %
Accuracy of the test images: 88 %
Accuracy of the test images: 85 %
Accuracy of the test images: 86 %
Accuracy of the test images: 86 %
Accuracy of the test images: 87 %
Accuracy of the test images: 86 %
Accuracy of the test images: 87 %
Accuracy of the test images: 87 %
Accuracy of the test images: 86 %
Accuracy of the test images: 87 %
Accuracy of the test images: 86 %
Accuracy of the test images: 86 %
Accuracy of the test images: 87 %
Accuracy of the test images: 87 %
Accuracy of the test images: 87 %
Accuracy of the test images: 86 %
Accuracy of the test images: 87 %
Accuracy of the test images: 87 %
Accuracy of the test images: 88 %
Accuracy of th

Accuracy of the test images: 87 %
Accuracy of the test images: 87 %
Accuracy of the test images: 87 %
Accuracy of the test images: 87 %
Accuracy of the test images: 87 %
Accuracy of the test images: 87 %
Accuracy of the test images: 87 %
Accuracy of the test images: 87 %
Accuracy of the test images: 87 %
Accuracy of the test images: 87 %
Accuracy of the test images: 87 %
Accuracy of the test images: 87 %
Accuracy of the test images: 88 %
Accuracy of the test images: 88 %
Accuracy of the test images: 88 %
Accuracy of the test images: 88 %
Accuracy of the test images: 88 %
Accuracy of the test images: 88 %
Accuracy of the test images: 88 %
Accuracy of the test images: 88 %
Accuracy of the test images: 88 %
Accuracy of the test images: 88 %
Accuracy of the test images: 88 %
Accuracy of the test images: 88 %
Accuracy of the test images: 88 %
Accuracy of the test images: 88 %
Accuracy of the test images: 88 %
Accuracy of the test images: 88 %
Accuracy of the test images: 88 %
Accuracy of th

[tensor([ 7,  2,  4,  0,  4,  1,  4,  9,  5,  9]), tensor([ 0,  6,  9,  0,  1,  8,  9,  7,  3,  4]), tensor([ 7,  6,  6,  8,  4,  0,  7,  4,  0,  1]), tensor([ 3,  1,  3,  4,  7,  2,  7,  1,  2,  1]), tensor([ 8,  7,  4,  2,  3,  5,  1,  2,  4,  4]), tensor([ 6,  3,  5,  8,  6,  0,  4,  1,  9,  5]), tensor([ 7,  8,  9,  2,  7,  9,  6,  4,  3,  0]), tensor([ 7,  0,  2,  8,  1,  7,  3,  2,  4,  7]), tensor([ 7,  6,  2,  7,  8,  4,  7,  3,  6,  1]), tensor([ 3,  6,  4,  3,  1,  4,  1,  7,  6,  9]), tensor([ 6,  0,  5,  4,  8,  9,  2,  6,  9,  4]), tensor([ 8,  2,  5,  9,  7,  9,  4,  4,  8,  2]), tensor([ 5,  4,  7,  6,  4,  9,  0,  5,  8,  5]), tensor([ 6,  6,  8,  7,  8,  1,  0,  1,  6,  4]), tensor([ 6,  7,  3,  1,  7,  1,  8,  2,  0,  2]), tensor([ 9,  9,  5,  8,  1,  5,  0,  0,  5,  4]), tensor([ 4,  6,  5,  4,  6,  5,  4,  5,  1,  4]), tensor([ 4,  9,  2,  3,  2,  7,  1,  8,  1,  8]), tensor([ 1,  8,  5,  0,  0,  4,  2,  5,  0,  1]), tensor([ 1,  1,  0,  9,  0,  8,  1,  6,  4,  2]),

[tensor([ 7,  2,  1,  0,  4,  1,  4,  9,  5,  9]), tensor([ 0,  6,  9,  0,  1,  5,  9,  7,  3,  4]), tensor([ 9,  6,  6,  5,  4,  0,  7,  4,  0,  1]), tensor([ 3,  1,  3,  4,  7,  2,  7,  1,  2,  1]), tensor([ 1,  7,  4,  2,  3,  5,  1,  2,  4,  4]), tensor([ 6,  3,  5,  5,  6,  0,  4,  1,  9,  5]), tensor([ 7,  8,  9,  3,  7,  4,  6,  4,  3,  0]), tensor([ 7,  0,  2,  9,  1,  7,  3,  2,  9,  7]), tensor([ 7,  6,  2,  7,  8,  4,  7,  3,  6,  1]), tensor([ 3,  6,  9,  3,  1,  4,  1,  7,  6,  9]), tensor([ 6,  0,  5,  4,  9,  9,  2,  1,  9,  4]), tensor([ 8,  7,  3,  9,  7,  4,  4,  4,  9,  2]), tensor([ 5,  4,  7,  6,  7,  9,  0,  5,  8,  5]), tensor([ 6,  6,  5,  7,  8,  1,  0,  1,  6,  4]), tensor([ 6,  7,  3,  1,  7,  1,  8,  2,  0,  2]), tensor([ 9,  9,  5,  5,  1,  5,  6,  0,  3,  4]), tensor([ 4,  6,  5,  4,  6,  5,  4,  5,  1,  4]), tensor([ 4,  7,  2,  3,  2,  7,  1,  8,  1,  8]), tensor([ 1,  8,  5,  0,  8,  9,  2,  5,  0,  1]), tensor([ 1,  1,  0,  9,  0,  3,  1,  6,  4,  2]),

In [0]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

 
# MNIST Dataset (Images and Labels)
training_dataset = dsets.MNIST(root ='./data',
                            train = True,
                            transform = transforms.ToTensor(),
                            download = True)
training_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = 10,
                                           shuffle = True)
testing_dataset = dsets.MNIST(root ='./data',
                           train = False,
                           transform = transform)
testing_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = 10,
                                          shuffle = False)
 

In [19]:
len(training_dataset)

60000